In [ ]:
from caveclient import CAVEclient
from intern import array
import pickle
import numpy as np
from agents import data_loader
from cloudvolume import CloudVolume
from membrane_detection import membranes
from agents.scripts import precompute_membrane_vectors, create_post_matrix, merge_paths, get_soma
import agents.sensor
from agents.run import run_agents
import aws.sqs as sqs
import sys
import time
import ast
import pandas as pd
import agents.scripts as scripts
from drive import drive
from finding_orphans import *
from math import sqrt
from tip_finding.tip_finding import tip_finder_decimation

In [2]:
load = pd.read_csv("agents/Data/endpoints_gt2.csv")
endpoints_dict = {}

In [ ]:
load.head()

In [5]:
#creating series objects for the desired collumns 
segIDs = load['seg_id']
Endpoints = load['Endpoints']


In [ ]:
for idx, i in enumerate(load['seg_id']):
    try:
        t1, skel, mesh_obj = tip_finder_decimation(str(i))
        endpoints_dict[i] = t1
    except:
        print(f"\n\nSeg {i} returned error on get. Skipping.\n")
        pass

load['Endpoints'] = load['Endpoints'].apply(lambda x: list(ast.literal_eval(x)))
load['proposed_endpoints'] = load.seg_id.map(endpoints_dict)

In [ ]:
#correctly  have 2 collumns for the proposed vs gt endpoints
load.head()

In [8]:

def find_euclidian_distance(proposed_endpoint, gt_endponts_array):
    smallest_distance = 2**30
    for endpoint in gt_endponts_array:
        difference = proposed_endpoint - endpoint
        distance = np.sqrt(np.sum(np.square(difference)))
        if(distance < smallest_distance):
            smallest_distance = distance 
    return smallest_distance 
            
    
     


In [50]:
def get_recall(num_correct, array_endpoints): 
    recall = num_correct / len(array_endpoints)
    return recall * 100

In [53]:
def get_precision(num_correct, array_proposed): 
    precision = num_correct / len(array_proposed)
    return precision * 100

In [ ]:
#TEST 1: invalidation_d at default: 12000, WITH Humfrey_Smoothing: 50 iterations, Decimation_Factor = 0.50

#should track total amount of correct gt recovered 

for index, row in load.iterrows():
    #reset num correct
    num_correct = 0
    threshold = 500
    endpoint_array = np.array(row["Endpoints"])
    proposed_endpoints_array = np.array(row["proposed_endpoints"])
    print("Endpoint Array:")
    print(endpoint_array)
    #print(len(endpoint_array))
    print()
    print("Proposed Endpoint Array:")
    print(proposed_endpoints_array)
    print()
    #print(load["Endpoints"])
   
    #if both say no endpoints, then it's correct
    if (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) == 0):
        print("correct, no endpoints")
        print("\nNEXT POINTS")
        continue
    #if we propose no endpoints but there are endpoints, it's wrong
    elif (len(proposed_endpoints_array.shape) == 0 and len(endpoint_array.shape) > 0):
        print("no proposed endpoints found, but ground truth endpoints exist")
        print("\nNEXT POINTS")
        continue 

    #if we propose endpoints but there are none, it's wrong
    elif proposed_endpoints_array.size > 0 and endpoint_array.size == 0:
        print("not correct--too many proposed endpoints but none ground truth endpoints exist")
        print("\nNEXT POINTS")
        continue
    #should skip for now if the endpoint array is size zero to avoid divide by zero error
    elif endpoint_array.size == 0:
        print("array size is 0")
        continue
    else:                                                                                   
        for proposed_endpoint in proposed_endpoints_array:
            smallest_distance= find_euclidian_distance(proposed_endpoint, endpoint_array)
            print(smallest_distance)
            
            
            #compare to threshold and return yes or no 
            if smallest_distance < threshold:
                print("correct")
                num_correct = num_correct + 1
                print()
            else:
                print("wrong")
                print()
        #get precision and recall at end of each iteration 
        print("recall = " + str(get_recall(num_correct, endpoint_array)))
        print("precision = " + str(get_precision(num_correct, proposed_endpoints_array)))
        print()
        print("\nNEXT POINTS")
        
        
    

In [ ]:
load.head()

make threshold for each endpoint
experiment with box (cannot overlap)
for each endpoint see if it finds in box -- if yes true found endpont, if false --not an endpoint
find distance from predicted to ground truth 
could still do MSE 

Threshold: 
-find the size of the segment (maybe connected componenets)
-array from the skel
-flat is edge case 
-maybe a percentage 
--a hard number 





In [33]:
if len(proposed_endpoints_array.shape) == 0:
    print("correct")


correct


PRECISION: #test at ground truth / # test  

RECALL: # gt recovered / number ground truth 
(would be 100% if you found all the correct endponts...NO EXTRAS)